In [1]:
import pandas as pd
import numpy as np
import json
from flatten_json import flatten
import time
import atexit
import asyncio
from pprint import pprint
from datetime import datetime
from datetime import date
from tda.client import Client
from tda.streaming import StreamClient
from tda.auth import easy_client
from tda import auth
from tda.orders import equities
from tda.orders.common import Duration, Session

In [2]:
# Set account variables and webdriver path
account_id = open('/mnt/c/Python/API Keys/TD/TD_ACCOUNT_ID.txt').read()
rothira_account_id = open('/mnt/c/Python/API Keys/TD/TD_ROTHIRA_ACCOUNT_ID.txt').read()
traditionalira_account_id = open('/mnt/c/Python/API Keys/TD/TD_TRADITIONALIRA_ACCOUNT_ID.txt').read()
consumer_key = open('/mnt/c/Python/API Keys/TD/TD_CONSUMER_KEY.txt').read()
redirect_uri = 'http://localhost'
token_path = '/mnt/c/Python/API Keys/TD/ameritrade-credentials.json'
geckodriver_path = '/usr/local/bin/geckodriver'
# Creates Webdriver for Selenium
def make_webdriver():
    # Import selenium here because it's slow to import
    from selenium import webdriver
    driver = webdriver.Firefox(executable_path = geckodriver_path)
    atexit.register(lambda: driver.quit())
    return driver

In [3]:
# Create client object
c = easy_client(consumer_key,
                            redirect_uri,
                            token_path,
                            make_webdriver)
streamc = StreamClient(c, account_id=int(account_id))

In [5]:
symbol_list = ['SPY', 'QQQ']

In [6]:
quote_response = c.get_quotes(symbol_list)
TDformatter().Quote_formatter(quote_response).to_df()

,assetType,assetMainType,cusip,assetSubType,symbol,description,bidPrice,bidSize,bidId,askPrice,...,regularMarketLastPrice,regularMarketLastSize,regularMarketNetChange,regularMarketTradeTimeInLong,netPercentChangeInDouble,markChangeInDouble,markPercentChangeInDouble,regularMarketPercentChangeInDouble,delayed,realtimeEntitled
SPY,ETF,EQUITY,78462F103,ETF,SPY,SPDR S&P 500,452.71,100,P,452.76,...,453.08,0,0.0,1630713600065,0.0,0.0,0.0,0.0,False,True
QQQ,ETF,EQUITY,46090E103,ETF,QQQ,"Invesco QQQ Trust, Series 1",381.42,200,P,381.45,...,381.57,1939,0.0,1630699200802,-0.0314,0.0,0.0,0.0,False,True


In [7]:
history_dict = {}
for symbol in symbol_list:
    history_response = c.get_price_history(symbol = symbol,
                                        period_type = Client.PriceHistory.PeriodType.YEAR,
                                        period = Client.PriceHistory.Period.TWENTY_YEARS,
                                        frequency_type = Client.PriceHistory.FrequencyType.DAILY,
                                        frequency = Client.PriceHistory.Frequency.DAILY)
    history_dict[symbol] = TDformatter.History_formatter(history_response).to_df()

In [8]:
chain_dict = {}
for symbol in symbol_list:
    chain_response = c.get_option_chain(symbol = symbol,
                      contract_type = Client.Options.ContractType.CALL,
                      strike_range = Client.Options.StrikeRange.ALL)
    chain_dict[symbol] = TDformatter().Chain_formatter(chain_response).to_df()

In [74]:
account_info = c.get_accounts(fields = [c.Account.Fields.POSITIONS, c.Account.Fields.ORDERS])
account_info_df = TDformatter.Account_formatter(account_info).to_df()

In [68]:
accounts = [account_id, rothira_account_id, traditionalira_account_id]
transaction_history = list(map(c.get_transactions, accounts))
transaction_history_df = TDformatter.Transaction_history_formatter(transaction_history).to_df()

In [147]:
watchlist = c.get_watchlists_for_multiple_accounts()
main_watchlist_df = TDformatter.Watchlist_formatter(watchlist, 'Main Watchlist').to_df()


In [148]:
main_watchlist_df

,sequenceId,quantity,averagePrice,commission,instrument.assetType
instrument.symbol,,,,,
AMD,1,0.0,0.0,0.0,EQUITY
MVST,2,0.0,0.0,0.0,EQUITY
PSTH,3,0.0,0.0,0.0,EQUITY
PLTR,4,0.0,0.0,0.0,EQUITY
NVDA,5,0.0,0.0,0.0,EQUITY
SPY,6,0.0,0.0,0.0,EQUITY
QQQ,7,0.0,0.0,0.0,EQUITY
AAPL,8,0.0,0.0,0.0,EQUITY
AMZN,9,0.0,0.0,0.0,EQUITY


In [5]:
## Check if Market is Open and Returns Bool ##
c.get_hours_for_single_market(c.Markets.EQUITY, datetime.now()).json()['equity']['equity']['isOpen']
c.get_hours_for_single_market(c.Markets.OPTION, datetime.now()).json()['option']['option']['isOpen']
# Checks /ES
c.get_hours_for_single_market(c.Markets.FUTURE, datetime.now()).json()['future']['ES']['isOpen']

True